In [1]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

submission = pd.read_csv('sample_submission.csv')

In [2]:
train.head()

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
0,0,5011000595017300,2871000192069300,음반,3
1,1,4148000690043300,5011000264024400,문화컨텐츠,3
2,2,5011000078068400,1120000007005400,농산물,3
3,3,4127100048006400,5011000587019400,기타식품,7
4,4,5011000078068400,2823700010076300,농산물,3


In [3]:
test.head()

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리
0,0,4167000577042200,5011000435014100,선케어
1,1,1156000009012200,5011000172034400,구강위생용품
2,2,4122000363057300,5011000361097300,캠핑
3,3,5011000436041400,2826000084036400,아웃도어가구
4,4,4150000241065200,5011000169044300,분유/이유식/아기간식


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31684 entries, 0 to 31683
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         31684 non-null  int64 
 1   송하인_격자공간고유번호  31684 non-null  int64 
 2   수하인_격자공간고유번호  31684 non-null  int64 
 3   물품_카테고리       31684 non-null  object
 4   운송장_건수        31684 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 1.2+ MB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7920 entries, 0 to 7919
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         7920 non-null   int64 
 1   송하인_격자공간고유번호  7920 non-null   int64 
 2   수하인_격자공간고유번호  7920 non-null   int64 
 3   물품_카테고리       7920 non-null   object
dtypes: int64(3), object(1)
memory usage: 247.6+ KB


In [6]:
train.isnull().sum()

index           0
송하인_격자공간고유번호    0
수하인_격자공간고유번호    0
물품_카테고리         0
운송장_건수          0
dtype: int64

## 데이터 전처리

In [7]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(train['물품_카테고리'])

train['물품_카테고리'] = encoder.transform(train['물품_카테고리'])
test['물품_카테고리'] = encoder.transform(test['물품_카테고리'])

In [8]:
category_columns = ['송하인_격자공간고유번호','수하인_격자공간고유번호', '물품_카테고리']
for col in category_columns:
  le = LabelEncoder()
  le.fit(pd.concat([train[col], test[col]], axis=0))
  train[col] = le.transform(train[col])
  test[col] = le.transform(test[col])

In [9]:
train.head()

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
0,0,3808,9092,67,3
1,1,1735,27887,34,3
2,2,2796,512,27,3
3,3,1266,30633,17,7
4,4,2796,8485,27,3


In [17]:
test.head()

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리
0,0,2085,30183,47
1,1,348,26920,12
2,2,1255,29164,88
3,3,3659,8899,58
4,4,1780,26855,39


## 모델링

In [30]:
train_X = train.drop(['index','운송장_건수'], axis = 1)
train_Y = train['운송장_건수']

test_X = test.drop(['index'],axis=1)

In [31]:
cat_features=['송하인_격자공간고유번호','수하인_격자공간고유번호','물품_카테고리']

In [44]:
#!pip install catboost

In [36]:
import catboost
from catboost import CatBoostRegressor

model = CatBoostRegressor(n_estimators=1000, learning_rate=0.01, cat_features=cat_features)
model.fit(train_X, train_Y, verbose=False)

In [37]:
predict = model.predict(test_X)
predict

array([6.90672253, 6.49788162, 5.01549554, ..., 3.66015156, 3.66015156,
       6.84743426])

In [38]:
submission['운송장_건수'] = predict
submission.to_csv('file.csv',index = False)